In [34]:
# classe fila
from tabulate import tabulate
class FilaSimples:

    def __init__(self, servidores, capacidade, min_max_temp_cheg, min_max_temp_serv, seeds) -> None:
        self.FILA = 0
        self.tempo = 0.0
        self.min_max_temp_cheg = min_max_temp_cheg
        self.min_max_temp_serv = min_max_temp_serv
        self.seeds = seeds
        self.seed = 0
        self.capacidade = capacidade
        self.servidores = servidores

    def escalonador(self, tempo_inicial):
        self.escalonador_eventos = []
        self.tabela_estados_fila = []
        # cria um vetor estado inicial com tamanho 3 + (capacidade + 1)
        estado = [0.0] * (3 + (self.capacidade + 1))
        estado[0] = None
        estado[1] = self.FILA
        estado[2] = self.tempo
        # novo evento sem sorteio
        evento = ['chegada', tempo_inicial, 0.0]
        self.escalonador_eventos.append(evento)
        self.tabela_estados_fila.append(estado)
        while self.seed < len(self.seeds):
            self.escalonador_eventos.sort( key=lambda x: x[1] )
            evento = self.escalonador_eventos.pop(0) 
            tempo_anterior = self.tempo
            self.tempo = evento[1]
            if evento[0] == 'chegada':
                if self.FILA < self.capacidade:
                    self.FILA += 1
                    if self.FILA <= self.servidores:
                        self.agendaSaida(self.tempo)
                self.agendaChegada(self.tempo)
            elif evento[0] == 'saida':
                self.FILA -= 1
                if self.FILA > 0:
                    self.agendaSaida(self.tempo)
            
            self.calculaEstado(evento, tempo_anterior)
        #print(tabulate(self.tabela_estados_fila))
        return self.tabela_estados_fila[-1]

    def calculaEstado(self, evento, tempo_anterior):
        ultimo_estado = self.tabela_estados_fila[-1]
        ultima_fila = ultimo_estado[1]
    
        novo_estado = ultimo_estado.copy()
        novo_estado[0] = evento[0]
        novo_estado[1] = self.FILA
        novo_estado[2] = self.tempo
        # diferenca entre tempo atual e o anterior
        diferenca = self.tempo - tempo_anterior
        
        # atualiza quanto tempo a fila ficou com a quantidade de pessoas anterior
        # diferenca + tempo anterior da fila com a quantidade de pessoas anterior
        novo_estado[3 + ultima_fila] = diferenca + ultimo_estado[3 + ultima_fila]
        
        self.tabela_estados_fila.append(novo_estado)

    def _sorteio(self, min_max):
        if self.seed == len(self.seeds):
            return None
        self.seed += 1
        return (min_max[1] - min_max[0]) * self.seeds[self.seed - 1] + min_max[0]
    
    def agendaChegada(self, tempo):
        sorteio = self._sorteio(self.min_max_temp_cheg)
        evento = ['chegada', tempo + sorteio, sorteio]
        self.escalonador_eventos.append(evento)
        
    def agendaSaida(self, tempo):
        sorteio = self._sorteio(self.min_max_temp_serv)
        evento = ['saida', tempo + sorteio, sorteio]
        self.escalonador_eventos.append(evento)

In [9]:
seeds = [0.3276, 0.8851, 0.1643, 0.5542, 0.6813, 0.7221, 0.9881]
min_max_tempo_chegada = [1, 2]
min_max_tempo_servico = [3, 6]

In [35]:
seeds = [0.3276, 0.8851, 0.1643, 0.5542, 0.6813]
min_max_tempo_chegada = [1, 3]
min_max_tempo_servico = [2, 4]

In [53]:
import numpy as np
simulados = []
for seed in seeds:
    iter = createSeeds(seed, 100000)
    fila = FilaSimples(1, 5, min_max_tempo_chegada, min_max_tempo_servico, iter)
    estados = fila.escalonador(3.0)
    simulados.append(estados[2:])
    
print(tabulate(simulados))
# Convert the data to a NumPy array
data_array = np.array(simulados)

# Calculate the mean of each column
column_means = np.mean(data_array, axis=0)
# Set the print options to format with 2 decimal places
np.set_printoptions(precision=4, suppress=True)
print(column_means)
print(tabulate(calculaResultados(column_means, 5)))

------  ------  ------  -------  -------  -------  -------
119786  3       3.2768   6.1816  131.403  42906.1  76736.1
120038  3       1.851    6.6982  149.827  42764.5  77111.8
119993  3.3144  7.3736   6.595   153.717  42907.6  76914.6
119828  3       6.0292  22.1003  171.096  43174.2  76451.8
119666  3       1.813    1.9054  114.977  42718.3  76826.1
------  ------  ------  -------  -------  -------  -------
[119862.2295      3.0629      4.0687      8.6961    144.2039  42894.1416
  76808.0563]
-  ------------  ------------
0       3.06288    0.00255533
1       4.06872    0.0033945
2       8.69609    0.00725507
3     144.204      0.120308
4   42894.1       35.7862
5   76808.1       64.0803
   119862        100
-  ------------  ------------


In [54]:
def calculaResultados(lista, capacidade):
    resultados = []
    for i in range(capacidade + 1):
        resultado = [i, lista[i + 1], (lista[i + 1] / lista[0]) * 100]
        resultados.append(resultado)
    resultados.append(['TOTAL', lista[0], 100])
    return resultados

In [29]:
def createSeeds(seed, n):
    seeds = []
    for i in range(n):
        seeds.append(seed)
        seed = (seed * 5) % 1
    return seeds